In [11]:
# Step 0. Load libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.formula.api as smf
import folium
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import  metrics

In [5]:
#https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv

df_raw = pd.read_csv('../data/raw/titanic_train.csv', index_col=0)


(891, 11)

In [6]:
df_raw.shape

(891, 11)

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [7]:
df = df_raw.copy()

Como existe desbalance me tengo que fijar más en las medidas de f1 y no tanto en accuracy

In [12]:
#Vemos si está desbalanceado
df['Survived'].value_counts()/len(df['Survived'])

0    0.616162
1    0.383838
Name: Survived, dtype: float64

Algoritmos que se podrían usar:
Decision tree, logistic regression, entre otros.
Algoritmos más complejos necesitan más datos.
Criterio para analizar random forest es más complejo para interpretar, deberíamos de analizar cada árbol y realizar el promedio.

In [13]:
#Todos los nombres son diferentes
df['Name'].nunique()

891

In [15]:
df=df.drop(columns=['Name'])

In [16]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [17]:
#cambiamos Sex por números
df['Sex']=df['Sex'].map({'male':1,'female':0})

In [20]:
df['Sex']

PassengerId
1      1
2      0
3      0
4      0
5      1
      ..
887    1
888    0
889    0
890    1
891    1
Name: Sex, Length: 891, dtype: int64

In [19]:
df.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
830,1,1,0,62.0,0,0,113572,80.0000,B28,NaN
295,0,3,1,24.0,0,0,349233,7.8958,NaN,S
380,0,3,1,19.0,0,0,347069,7.7750,NaN,S
569,0,3,1,NaN,0,0,2686,7.2292,NaN,C
259,1,1,0,35.0,0,0,PC 17755,512.3292,NaN,C
869,0,3,1,NaN,0,0,345777,9.5000,NaN,S
509,0,3,1,28.0,0,0,C 4001,22.5250,NaN,S
10,1,2,0,14.0,1,0,237736,30.0708,NaN,C
319,1,1,0,31.0,0,2,36928,164.8667,C7,S


In [22]:
df['Ticket'].nunique()

681

In [23]:
df=df.drop(columns=['Ticket'])

In [26]:
df['Cabin'].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

In [27]:
df['Cabin'].nunique()

147

In [28]:
df=df.drop(columns=['Cabin'])

In [29]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [30]:
df['Embarked']=df['Embarked'].map({'S':2,'C':1,'Q':0})

In [31]:
df['Embarked']

PassengerId
1      2.0
2      1.0
3      2.0
4      2.0
5      2.0
      ... 
887    2.0
888    2.0
889    2.0
890    1.0
891    0.0
Name: Embarked, Length: 891, dtype: float64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 62.6 KB


In [42]:
mean_age = df['Age'].mean()
print(mean_age)

df['Age'][np.isnan(df['Age'])] = mean_age


29.69911764705882


/tmp/ipykernel_6015/382355856.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'][np.isnan(df['Age'])] = mean_age


In [43]:
df['Embarked'].value_counts()

2.0    644
1.0    168
0.0     77
Name: Embarked, dtype: int64

In [44]:
df['Embarked'][np.isnan(df['Embarked'])] = 2

/tmp/ipykernel_6015/2528881676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][np.isnan(df['Embarked'])] = 2


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 62.6 KB


In [46]:
#Dividimos el dataset
X=df.drop(columns=['Survived'])
y=df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=520, test_size=0.2) #darle menos datos de entrenamiento para que no haya overfit
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=520, test_size=0.3)

leakage: nivel de correlación de las variables

In [53]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,0,7.2500,2.0
2,1,0,38.0,1,0,71.2833,1.0
3,3,0,26.0,0,0,7.9250,2.0
4,1,0,35.0,1,0,53.1000,2.0
5,3,1,35.0,0,0,8.0500,2.0


In [54]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [38]:
#nos da los parámetros
#RandomForestClassifier?


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [47]:
modelo=RandomForestClassifier(n_estimators=50, random_state=145) # Le damos menos árboles para que no haya overfit

In [48]:
modelo.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50, random_state=145)

In [49]:
y_train_pred =modelo.predict(X_train)
y_test_pred =modelo.predict(X_test)

In [51]:
#Usamos una métrica para ver las diferencias

train_report =metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       456
           1       0.98      0.97      0.98       256

    accuracy                           0.98       712
   macro avg       0.98      0.98      0.98       712
weighted avg       0.98      0.98      0.98       712



In [52]:
test_report =metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.78      0.90      0.84        93
           1       0.87      0.72      0.79        86

    accuracy                           0.82       179
   macro avg       0.83      0.81      0.81       179
weighted avg       0.82      0.82      0.81       179



f1 score del train es 1 y del test es 0.76, al algoritmo esta memorizando la data.

CAMBIAMOS LOS DATOS PARA AJUSTAR

In [56]:
modelo=RandomForestClassifier(n_estimators=5, random_state=42, max_leaf_nodes=3)

In [57]:
modelo.fit(X_train,y_train)

RandomForestClassifier(n_estimators=20, random_state=145)

In [58]:
y_train_pred =modelo.predict(X_train)
y_test_pred =modelo.predict(X_test)

In [61]:
train_report =metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       456
           1       0.97      0.96      0.97       256

    accuracy                           0.98       712
   macro avg       0.98      0.97      0.98       712
weighted avg       0.98      0.98      0.98       712



In [62]:
test_report =metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.77      0.88      0.82        93
           1       0.85      0.72      0.78        86

    accuracy                           0.80       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.80      0.80       179

